In [1]:
const output = "/tmp/test.dylib"

"/tmp/test.dylib"

In [2]:
cprog = """
#include <stdlib.h>

typedef struct 
{
    int a;
    double b;
} FOO, *FOOP;

FOO test1() {
    FOO x;
    x.a = 1;
    x.b = 2.0;
    return x;
}

FOO test2(FOO x) {
    return x;
}

FOO test3(FOOP xp) {
    FOO y;
    y.a = xp->a;
    y.b = xp->b;
    return y;
}

FOOP test4(FOOP xp) {
    FOOP yp = (FOOP) malloc(sizeof(FOO));
    yp->a = xp->a;
    yp->b = xp-> b;
    return yp;
}
""";

In [3]:
open(`gcc -fPIC -O3 -xc -shared -o $output -`, "w") do f
     print(f, cprog)
end

In [4]:
struct FOO
    a::Cint
    b::Cdouble
end

In [9]:
x = ccall((:test1, output), FOO, ())

FOO(1, 2.0)

In [11]:
x = ccall((:test2, output), FOO, (FOO,), x)

FOO(1, 2.0)

In [15]:
p = ccall((:test3, output), FOO, (Ptr{FOO},), Ref(x))

FOO(1, 2.0)

In [14]:
p = ccall((:test3, output), FOO, (Ptr{FOO},), pointer_from_objref(x))

FOO(1, 2.0)

In [17]:
q = ccall((:test4, output), Ptr{FOO}, (Ptr{FOO},), Ref(x))

Ptr{FOO} @0x00007f95ca1d8fc0

In [18]:
unsafe_load(q)

FOO(1, 2.0)

In [19]:
bar(xr::Ptr{FOO}) = ccall((:test4, output), Ptr{FOO}, (Ptr{FOO},), xr)

bar (generic function with 1 method)

In [20]:
bar(Ref(x))

LoadError: [91mMethodError: no method matching bar(::Base.RefValue{FOO})[0m
Closest candidates are:
  bar([91m::Ptr{FOO}[39m) at In[19]:1[39m

In [22]:
bar(convert(Ptr{FOO}, pointer_from_objref(x)))

Ptr{FOO} @0x00007f95c9fefa10